## 1. Importar Librerías Necesarias

Importamos las bibliotecas necesarias:
- `os`: Para acceder a variables de entorno
- `dotenv`: Para cargar configuración desde archivo .env
- `openai`: SDK de Azure OpenAI para Python

In [ ]:
import os
from dotenv import load_dotenv
import openai

## 2. Cargar Variables de Entorno

Cargamos las credenciales y configuración desde el archivo `.env`:
- `AZURE_OPENAI_ENDPOINT`: URL del servicio de Azure OpenAI
- `AZURE_OPENAI_APIKEY`: Clave de acceso al servicio
- `AZURE_OPENAI_VERSION`: Versión de la API
- `MODEL`: Nombre del modelo a utilizar

In [ ]:
# Cargar variables de entorno desde .env
load_dotenv()

print(f"✅ Configuración cargada")
print(f"Modelo a utilizar: {os.getenv('MODEL')}")

## 3. Definir Textos de Prueba

Creamos una lista de textos que contienen fechas en **diferentes formatos**:

### Formatos de fecha incluidos:
- **ISO 8601**: `YYYY-MM-DD` (ej: 2023-04-15)
- **Formato americano**: `MM/DD/YYYY` (ej: 10/15/2023)
- **Formato europeo con guiones**: `DD-MM-YYYY` (ej: 11-01-2023)
- **Texto natural**: "23 de octubre de 2023", "1 de agosto de 2023"
- **Rangos de fechas**: "del 11-01-2023 al 11-05-2023"

Estos textos simulan casos reales de documentos, emails, reportes, etc.

In [ ]:
text_list = [
    "La conferencia de tecnología se llevará a cabo el 2023-04-15 en el centro de convenciones de la ciudad.",
    "La temporada cultural de otoño arranca con una serie de eventos imperdibles. El primero será la exposición de arte moderno que abre sus puertas el 10/15/2023 en el Museo de la Ciudad. A continuación, el 23 de octubre de 2023, se presentará la aclamada obra de teatro 'Luces de Bohemia' en el Teatro Principal. Finalmente, no te pierdas la feria del libro que se realizará del 11-01-2023 al 11-05-2023, donde podrás encontrar las últimas publicaciones y conocer a tus autores favoritos.",
    "El proyecto de desarrollo de software se desglosa en varias fases clave con fechas de entrega específicas. La fase de investigación debe completarse antes del 2023-05-20. La etapa de diseño y prototipado está programada para el período comprendido entre el 01/06/2023 y el 31/07/2023. La implementación del código comenzará el 1 de agosto de 2023 y se extenderá hasta el 30 de septiembre de 2023. Por último, la fase de pruebas y ajustes se llevará a cabo del 10-10-2023 al 10-12-2023, asegurando que el producto final esté listo para su lanzamiento el 15 de diciembre de 2023.",
    "El vuelo está programado para despegar a las 14:30 del 05-06-2023 desde el aeropuerto internacional.",
    "La videoconferencia internacional se iniciará a las 09:00 AM GMT del 22 de noviembre de 2023."
]

print(f"📋 Total de textos a procesar: {len(text_list)}")
print(f"\n📄 Ejemplo del primer texto:")
print(f"   {text_list[0]}")

## 4. Función de Extracción de Fechas

Esta función implementa el **patrón few-shot learning**:

### ¿Qué es Few-Shot Learning?
Es una técnica donde proporcionamos al modelo uno o más ejemplos de entrada-salida para guiar su comportamiento.

### Estructura de la función:
1. **System Prompt**: Define el rol y comportamiento del asistente
2. **Ejemplo (shot)**: 
   - User: "La reunión es el 3 de enero de 2024 y la entrega es el 12 de abril de 2024."
   - Assistant: "03/01/2024, 12/04/2024"
3. **Consulta real**: El texto del que queremos extraer fechas

### Formato de salida esperado:
- Todas las fechas en formato `dd/mm/aaaa`
- Separadas por comas
- Sin texto adicional

In [ ]:
def extract_dates_from_text(text):
    """
    Extrae todas las fechas de un texto y las normaliza al formato dd/mm/aaaa.
    
    Args:
        text (str): Texto del cual extraer fechas
        
    Returns:
        str: Fechas extraídas en formato dd/mm/aaaa, separadas por comas
    """
    # Crear cliente de Azure OpenAI
    openai_client = openai.AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_APIKEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
    )

    # Definir el rol del asistente
    system_prompt = "Eres un asistente que extrae fechas de textos. Responde únicamente con las fechas en formato dd/mm/aaaa"
    
    # Construir el prompt del usuario
    user_prompt = f'''Extrae todas las fechas del siguiente texto: {text}

    Response:
    [
    '''

    # Realizar la llamada al modelo con few-shot learning
    response = openai_client.chat.completions.create(
        model=os.getenv("MODEL"),
        messages=[
            {"role": "system", "content": system_prompt},
            # Few-shot example (ejemplo de entrada-salida)
            {"role": "user", "content": "Extrae todas las fechas del siguiente texto: La reunión es el 3 de enero de 2024 y la entrega es el 12 de abril de 2024."},
            {"role": "assistant", "content": "03/01/2024, 12/04/2024"},
            # Consulta real
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content

print("✅ Función extract_dates_from_text() definida")

## 5. Procesar Todos los Textos

Ahora ejecutamos la función para cada texto de la lista y mostramos:
- El número del texto
- Un fragmento del texto original (primeros 100 caracteres)
- Las fechas extraídas y normalizadas

**Nota**: Esto hará 5 llamadas a la API de Azure OpenAI.

In [ ]:
print("\n" + "="*80)
print("EXTRACCIÓN DE FECHAS")
print("="*80)
print(f"Usando el modelo: {os.getenv('MODEL')}\n")

# Iterar sobre cada texto y extraer fechas
for i, text_with_dates in enumerate(text_list, start=1):
    print(f"\n{'─'*80}")
    print(f"📝 TEXTO {i}")
    print(f"{'─'*80}")
    
    # Mostrar fragmento del texto original
    preview = text_with_dates[:100] + "..." if len(text_with_dates) > 100 else text_with_dates
    print(f"Original: {preview}")
    
    # Extraer fechas
    print(f"\n⏳ Extrayendo fechas...")
    dates = extract_dates_from_text(text_with_dates)
    
    # Mostrar resultado
    print(f"✅ Fechas extraídas: {dates}")

print(f"\n{'='*80}")
print("PROCESO COMPLETADO")
print("="*80)

## 6. Análisis de Resultados

Veamos qué tipos de fechas pudo identificar el modelo:

In [ ]:
print("\n📊 ANÁLISIS DE FORMATOS PROCESADOS:\n")
print("✓ Formato ISO (YYYY-MM-DD): 2023-04-15 → dd/mm/aaaa")
print("✓ Formato americano (MM/DD/YYYY): 10/15/2023 → dd/mm/aaaa")
print("✓ Formato europeo (DD-MM-YYYY): 11-01-2023 → dd/mm/aaaa")
print("✓ Texto natural: '23 de octubre de 2023' → dd/mm/aaaa")
print("✓ Texto natural: '1 de agosto de 2023' → dd/mm/aaaa")
print("✓ Con hora: '05-06-2023' (ignora la hora 14:30) → dd/mm/aaaa")
print("\n💡 El modelo LLM entiende el contexto y normaliza automáticamente!")

## Conclusión

Este ejemplo demuestra varios conceptos importantes:

### 1. **Few-Shot Learning**
- Proporcionamos un ejemplo al modelo para guiar su comportamiento
- El modelo aprende el patrón de extracción y formato deseado
- No necesitamos entrenar un modelo específico

### 2. **Normalización de Datos**
- Convertimos fechas de múltiples formatos a uno estándar
- Útil para integración con bases de datos
- Facilita el procesamiento posterior

### 3. **Procesamiento de Lenguaje Natural**
- El modelo entiende fechas escritas en texto natural
- Reconoce meses en español ("octubre", "agosto", "noviembre")
- Maneja diferentes formatos de escritura

### 4. **Extracción de Información**
- Identificamos datos específicos en textos complejos
- Procesamos múltiples fechas en un mismo texto
- Ignoramos información irrelevante (horas, zonas horarias)

### Aplicaciones Prácticas:

✅ **Procesamiento de documentos**: Extraer fechas importantes de contratos, reportes, etc.

✅ **Análisis de emails**: Identificar fechas de reuniones, deadlines, eventos

✅ **Migración de datos**: Normalizar fechas antes de importar a bases de datos

✅ **Automatización**: Crear recordatorios o calendarios automáticamente

✅ **Compliance**: Identificar fechas relevantes en documentos legales

### Ventajas vs Expresiones Regulares:

| Aspecto | Regex | LLM |
|---------|-------|-----|
| Formatos variados | ❌ Necesita patrones por cada formato | ✅ Entiende múltiples formatos |
| Texto natural | ❌ Muy difícil | ✅ Natural |
| Contexto | ❌ No entiende contexto | ✅ Usa contexto |
| Idiomas | ❌ Un patrón por idioma | ✅ Multilingüe |
| Mantenimiento | ❌ Complejo | ✅ Simple |

### Consideraciones:

⚠️ **Costo**: Cada llamada a la API tiene un costo

⚠️ **Latencia**: Más lento que regex para volúmenes muy grandes

⚠️ **Validación**: Siempre verifica los resultados críticos

✅ **Precisión**: Muy alta con prompts bien diseñados